In [2]:
import pandas as pd
import numpy as np

In [3]:
cols = (list(range(-1, 784)))
def prepare_ds(path, split=False, ratio=0.0):
    data = pd.read_csv(filepath_or_buffer=path, names=cols)
    data = data.sample(frac=1)
    if split == True:    
        d1X = np.array(data[:int(-data.shape[0] * ratio)][cols[1:]]).T
        d1Y = np.array(pd.get_dummies(data[:int(-data.shape[0] * ratio)][-1])[:]).T
        d2X = np.array(data[int(-data.shape[0] * ratio):][cols[1:]]).T
        d2Y = np.array(pd.get_dummies(data[int(-data.shape[0] * ratio):][-1])[:]).T
        return d1X, d1Y, d2X, d2Y
    if split == False:
        d1X = np.array(data[:][cols[1:]]).T
        d1Y = np.array(pd.get_dummies(data[:][-1])[:]).T
        return d1X, d1Y

In [4]:
data_path = '../digit-recognizer/mnist_train.csv'
test_path = '../digit-recognizer/mnist_test.csv'
train_X, train_Y, valid_X, valid_Y = prepare_ds(data_path, split=True, ratio=0.2)
test_X, test_Y, = prepare_ds(test_path)

In [5]:
m_train = train_X.shape[1]
m_test = valid_X.shape[1]
train_X.shape, train_Y.shape, valid_X.shape, valid_Y.shape

((784, 48000), (10, 48000), (784, 12000), (10, 12000))

In [6]:
def normalize(X, Y):
    X = X / 255
    return X, Y

In [7]:
def compute_LogLoss(Yp, Yg):
    cost = -(Yg * np.log(Yp) + (1 - Yg) * np.log(1 - Yp))
    return np.mean(cost)

In [8]:
def forward(W, b, X, eval=False):
    Z = np.dot(W.T, X) + b
    A = 1. / (1 + np.exp(-Z))
    if eval == False:
        return A
    if eval == True:
        return np.argmax(A, axis=0)

In [9]:
def backward(Yp, Yg, W, b, X, learning_rate=0.1):
    cost = compute_LogLoss(Yp.T, Yg.T)
    diff = Yp - Yg
    dW = np.dot(X, diff.T) / m_train
    db = np.sum(diff, axis=1, keepdims=True) / m_train
    
    W -= dW * learning_rate
    b -= db * learning_rate
    return cost, W, b

In [10]:
def accuracy(y_pred, y_actu):
    y_actt = np.argmax(y_actu, axis=0)
    return np.mean((y_pred == y_actt))

In [14]:
W = np.zeros((784, 10))
b = np.zeros((10, 1))
prep_X, prep_Y = normalize(train_X, train_Y)
for i in range(1000):
    pred_Y = forward(W, b, prep_X)
    cost, W, b = backward(pred_Y, prep_Y, W, b, prep_X)
    if i % 50 == 0:
        tr_X, tr_Y = prep_X, prep_Y
        vl_X, vl_Y = normalize(valid_X, valid_Y)
        tr_P = forward(W, b, tr_X, eval=True)
        vl_P = forward(W, b, vl_X, eval=True)

        print(i, cost, accuracy(tr_P, tr_Y), accuracy(vl_P, vl_Y) )

0 0.6931471805599451 0.6709166666666667 0.6749166666666667
50 0.16551267197030586 0.8286666666666667 0.8325
100 0.1350302890505606 0.8500833333333333 0.8524166666666667
150 0.12123033056985097 0.8589583333333334 0.8615
200 0.11298046408833355 0.8659791666666666 0.8679166666666667
250 0.10735783236621234 0.8702291666666667 0.8725
300 0.10321661540619932 0.8744583333333333 0.8751666666666666
350 0.10000426528760976 0.877 0.8775
400 0.09741790169046267 0.8795416666666667 0.8795
450 0.09527623444128704 0.8821458333333333 0.88175
500 0.09346350286445482 0.8842291666666666 0.8833333333333333
550 0.09190201062085201 0.8860208333333334 0.8838333333333334
600 0.09053749239605517 0.8875416666666667 0.88525
650 0.08933079178810324 0.8887291666666667 0.8861666666666667
700 0.08825287384439176 0.8898958333333333 0.8870833333333333
750 0.08728170426953134 0.8908958333333333 0.88825
800 0.0864002245957031 0.8919583333333333 0.8894166666666666
850 0.08559499717171773 0.8928125 0.8905833333333333
900 0

In [15]:
val_prep_X, val_prep_Y = normalize(valid_X, valid_Y)
valid_preds = forward(W, b, val_prep_X, eval=True)
accuracy(valid_preds, val_prep_Y)

0.8925833333333333

In [16]:
test_prep_X, test_prep_Y = normalize(test_X, test_Y)
test_preds = forward(W, b, test_prep_X, eval=True)
accuracy(test_preds, test_prep_Y)

0.9006